In [ ]:
import plotly.graph_objects as go
import numpy as np
from pathlib import Path
from imageio.v3 import imread

fx, fy, cx1, cy = 1774.02759117584, 1774.12812333568, 689.047651992725, 536.130613432710
cx2 = 703.759113246139
baseline = 40.42720433134619  # in millimeters

# 打印结果
print(f"fx: {fx}")
print(f"fy: {fy}")
print(f"cx1: {cx1}")
print(f"cy: {cy}")
print(f"cx2: {cx2}")
print(f"baseline: {baseline} mm")

disp_path = f"/Users/xiejunfeng/开心摸鱼项目/双目测距工程/MoChaOutputs/outputs/rectify_600.npy"
disp = np.load(disp_path)
image = imread("/Users/xiejunfeng/开心摸鱼项目/双目测距工程/RedTilmpa/rectified_img1_color_600.png")

# inverse-project
depth = (fx * baseline) / (-disp + (cx2 - cx1))
H, W = depth.shape
xx, yy = np.meshgrid(np.arange(W), np.arange(H))
points_grid = np.stack(((xx - cx1) / fx, (yy - cy) / fy, np.ones_like(xx)), axis=0) * depth

mask = np.ones((H, W), dtype=bool)

# Remove flying points
mask[1:][np.abs(depth[1:] - depth[:-1]) > 1] = False
mask[:, 1:][np.abs(depth[:, 1:] - depth[:, :-1]) > 1] = False

points = points_grid.transpose(1, 2, 0)[mask]
colors = image[mask].astype(np.float64) / 255

In [ ]:
NUM_POINTS_TO_DRAW = 200000

subset = np.random.choice(points.shape[0], size=(NUM_POINTS_TO_DRAW,), replace=False)
points_subset = points[subset]
colors_subset = colors[subset]

print("""
Controls:
---------
Zoom:      Scroll Wheel
Translate: Right-Click + Drag
Rotate:    Left-Click + Drag
""")

x, y, z = points_subset.T

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x, y=y, z=z,  # flipped to make visualization nicer
            mode='markers',
            marker=dict(size=1, color=colors_subset)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=True),
            yaxis=dict(visible=True),
            zaxis=dict(visible=True),
        )
    )
)

# 保存图表为HTML文件
output_file = "Mocha_rectify_600.html"
fig.write_html(output_file)
print(f"3D point cloud has been saved to {output_file}")

fig.show()

In [ ]:
# disp_path = f"/Users/xiejunfeng/开心摸鱼项目/双目测距工程/MoChaOutputs/outputs/origin_600.npy"
# disp = np.load(disp_path)
# image = imread("/Users/xiejunfeng/Pictures/test_红罗非鱼/Left/Image_600_left.png")
#
# # inverse-project
# depth = (fx * baseline) / (-disp + (cx2 - cx1))
# H, W = depth.shape
# xx, yy = np.meshgrid(np.arange(W), np.arange(H))
# points_grid = np.stack(((xx - cx1) / fx, (yy - cy) / fy, np.ones_like(xx)), axis=0) * depth
#
# mask = np.ones((H, W), dtype=bool)
#
# # Remove flying points
# mask[1:][np.abs(depth[1:] - depth[:-1]) > 1] = False
# mask[:, 1:][np.abs(depth[:, 1:] - depth[:, :-1]) > 1] = False
#
# points = points_grid.transpose(1, 2, 0)[mask]
# colors = image[mask].astype(np.float64) / 255
# NUM_POINTS_TO_DRAW = 200000
#
# subset = np.random.choice(points.shape[0], size=(NUM_POINTS_TO_DRAW,), replace=False)
# points_subset = points[subset]
# colors_subset = colors[subset]
#
# print("""
# Controls:
# ---------
# Zoom:      Scroll Wheel
# Translate: Right-Click + Drag
# Rotate:    Left-Click + Drag
# """)
#
# x, y, z = points_subset.T
#
# fig = go.Figure(
#     data=[
#         go.Scatter3d(
#             x=x, y=y, z=z,  # flipped to make visualization nicer
#             mode='markers',
#             marker=dict(size=1, color=colors_subset)
#         )
#     ],
#     layout=dict(
#         scene=dict(
#             xaxis=dict(visible=True),
#             yaxis=dict(visible=True),
#             zaxis=dict(visible=True),
#         )
#     )
# )
#
# # 保存图表为HTML文件
# output_file = "Mocha_origin_600.html"
# fig.write_html(output_file)
# print(f"3D point cloud has been saved to {output_file}")
#
# fig.show()
